# **Bangkit Final Project: World Coin Classification**

# **Dependencies**

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import zipfile

from tensorflow.keras import backend
from tensorflow.keras.models import Sequential, Model, model_from_json, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, LeakyReLU, Input
from tensorflow.keras.layers import GlobalAveragePooling2D, Lambda, Reshape, Concatenate, Average
from tensorflow.keras.applications import Xception
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.regularizers import Regularizer, l2

from google.colab import files, drive

In [ ]:
# Upload the kaggle.json file from Kaggle account settings page
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
# Install the Kaggle API client
!pip install -q kaggle

In [ ]:
# The Kaggle API client expects this file to be in ~/.kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup
!chmod 600 ~/.kaggle/kaggle.json

# **Importing Dataset**

In [ ]:
# Download the dataset
!kaggle datasets download -d wanderdust/coin-images

 99% 454M/459M [00:05<00:00, 65.0MB/s]
100% 459M/459M [00:06<00:00, 79.8MB/s]


In [ ]:
# If kaggle is down, mount from drive
try:
    coin_file = open('/content/coin-images.zip', 'r')
    filepath = '/content/coin-images.zip'
except FileNotFoundError:
    drive.mount('/content/drive')
    filepath = '/content/drive/My Drive/Bangkit project/Dataset/coin-images.zip'

# Unzip the dataset into folder
zip_ref = zipfile.ZipFile(filepath, 'r')
zip_ref.extractall('/content/')
zip_ref.close()

# **Data Preparation**

In [ ]:
# Define directories
data_dir = "/content/coins/data/"

train_dir = data_dir + "train/"
validation_dir = data_dir + "validation/"
test_dir = data_dir + "test/"

In [ ]:
# Create generators
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=360,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      brightness_range=[0.8,1.2],
      horizontal_flip=False,
      vertical_flip=False,
      featurewise_std_normalization=False,
      featurewise_center=False,
      samplewise_std_normalization=False,
      samplewise_center=False,
      fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=360,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      brightness_range=[0.8,1.2],
      horizontal_flip=False,
      vertical_flip=False,
      featurewise_std_normalization=False,
      featurewise_center=False,
      samplewise_std_normalization=False,
      samplewise_center=False,
      fill_mode='nearest'
)

test_datagen = ImageDataGenerator(
      rescale=1./255,
      featurewise_std_normalization=False,
      featurewise_center=False,
      samplewise_std_normalization=False,
      samplewise_center=False
)

In [ ]:
# Read images from generators
batch_size = 32
image_width = 299
image_height = 299

train_generator = train_datagen.flow_from_directory(
      train_dir,
      target_size=(image_width, image_height),
      class_mode="categorical",
      batch_size=batch_size
)

validation_generator = validation_datagen.flow_from_directory(
      validation_dir,
      target_size=(image_width, image_height),
      class_mode="categorical",
      batch_size=batch_size
)

test_generator = test_datagen.flow_from_directory(
      test_dir,
      target_size=(image_width, image_height),
      class_mode="categorical",
      batch_size=1
)

Found 6413 images belonging to 211 classes.
Found 844 images belonging to 211 classes.
Found 844 images belonging to 211 classes.


# **Model**

In [ ]:
drive.mount('/content/drive')
model_path = '/content/drive/My Drive/Bangkit project/models/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [20]:
# Load xception model
xception_model = load_model(model_path + "xception_model.h5")
xception_model = Model(inputs=xception_model.inputs, outputs=xception_model.outputs, name='xception')

# Load bilinear xception model
bilinear_xception_model = load_model(model_path + "bilinear_xception_model.h5")
bilinear_xception_model = Model(inputs=bilinear_xception_model.inputs,
                                outputs=bilinear_xception_model.outputs,
																name='bilinear_xception')

# Load inception model
inception_model = load_model(model_path + "inception_model.h5")
inception_model = Model(inputs=inception_model.inputs,
                                outputs=inception_model.outputs,
																name='inception')

models = [xception_model, inception_model, bilinear_xception_model]

In [21]:
model_input = Input(shape=(image_width, image_height, 3))
model_outputs = [model(model_input) for model in models] 

ensemble_output = Average()(model_outputs)

ensemble_model = Model(inputs=model_input, outputs=ensemble_output, name='ensemble')
ensemble_model.summary()

Model: "ensemble"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
xception (Model)                (None, 211)          22018811    input_2[0][0]                    
__________________________________________________________________________________________________
inception (Model)               (None, 211)          22960115    input_2[0][0]                    
__________________________________________________________________________________________________
bilinear_xception (Model)       (None, 211)          55837563    input_2[0][0]                    
___________________________________________________________________________________________

In [22]:
# Save model
ensemble_model.save("ensemble_model.h5")

In [23]:
ensemble_model.compile(loss='categorical_crossentropy', optimizer=Nadam(lr=0.0001), metrics=['accuracy'])

# Evaluate model
train_score = ensemble_model.evaluate(train_generator, verbose=1)
print("Training loss: ", train_score[0])
print("Training accuracy: ", train_score[1])

validation_score = ensemble_model.evaluate(validation_generator, verbose=1)
print("Validation loss: ", validation_score[0])
print("Validation accuracy: ", validation_score[1])

test_score = ensemble_model.evaluate(test_generator, verbose=1)
print("Testing loss: ", test_score[0])
print("Testing accuracy: ", test_score[1])

201/201 [==============================] - 179s 891ms/step - loss: 0.0559 - accuracy: 0.9853
Training loss:  0.05589056760072708
Training accuracy:  0.985342264175415
27/27 [==============================] - 23s 838ms/step - loss: 0.5810 - accuracy: 0.8531
Validation loss:  0.5809820890426636
Validation accuracy:  0.8530805706977844
844/844 [==============================] - 30s 35ms/step - loss: 0.4940 - accuracy: 0.8803
Testing loss:  0.49398916959762573
Testing accuracy:  0.8803317546844482


In [24]:
# Predict test images
predictions = []

for filename in test_generator.filenames:
    img = load_img(test_dir+filename, target_size=(image_width, image_height))
    img = img_to_array(img)/255
    img_expand = np.expand_dims(img, axis=0)
    predictions.append(ensemble_model.predict(img_expand)[0])

In [25]:
# Get index of largest probability
predicted_indices = np.argmax(predictions, axis=1)

# Get coin directory name from index 
directories = dict((v, k) for k, v in train_generator.class_indices.items())
predicted_dir = [directories.get(k) for k in predicted_indices]

# Get label name from coin directory name
with open(data_dir + 'cat_to_name.json', 'r') as json_file:
    labels = json.load(json_file)
predicted_labels = [labels.get(str(k)) for k in predicted_dir]

In [26]:
# Save predicted labels as CSV file
filenames = test_generator.filenames
results = pd.DataFrame({"Filename": filenames, "Predictions": predicted_labels})
results.to_csv("ensemble_results.csv", index=False)
results.head()

,Filename,Predictions
0,1/021__1 Cent_australia.jpg,"1 Cent,Australian dollar,australia"
1,1/022__1 Cent_australia.jpg,"1 Cent,Australian dollar,australia"
2,1/027__1 Cent_australia.jpg,"1 Cent,Australian dollar,australia"
3,1/036__1 Cent_australia.jpg,"1 Cent,Australian dollar,australia"
4,10/005__5 Centavos_brazil.jpg,"5 Centavos,Brazilian Real,brazil"


# **Convert to TFLite**

In [27]:
# Create converter
converter = tf.lite.TFLiteConverter.from_keras_model(ensemble_model)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
                                       tf.lite.OpsSet.SELECT_TF_OPS]
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Convert the model
tflite_model = converter.convert()
open("ensemble_model.tflite", "wb").write(tflite_model)

100952064

# **Copy model to Drive**

In [28]:
drive.mount('/content/drive')
!cp ensemble_model.h5 "/content/drive/My Drive/Bangkit project/models"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
